<a href="https://colab.research.google.com/github/lephuocdat2000/truyvandaphuongtien/blob/master/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import bs4
import pandas as pd
import requests
import re
import string
from datetime import date

In [42]:
#truy vấn nội dung HTML của một link
def get_page_content(url):
    page=requests.get(url)
    return bs4.BeautifulSoup(page.text,"html.parser")
#tách xâu ngày ra khỏi thẻ tag HTML
def get_date(date_tag):
    date=""
    str_date_tag=str(date_tag)
    date=str_date_tag[str_date_tag.find(">")+1:str_date_tag.find("<",1)]
    return date
#hàm so sánh xem ngày viết của bài báo có giới hạn trong vòng một tháng so với ngày hiện tại hay không
def date_comparing(today,date_article):
    day_article=date_article[date_article.find(",")+2:date_article.find("/")]
    month_article=str()
    year_article=str()
    for i in date_article[date_article.find("/")+1:date_article.find("/")+3]:
        if (i!="/"): month_article+=i
    for i in date_article[date_article.find("/")+3:date_article.find("/")+8]:
        if (i!="/" and i!=','): year_article+=i
    day_article=int(day_article)
    month_article=int(month_article)
    year_article=int(year_article)
    if (today.year-1==year_article) and (today.month==1) and (month_article==12) and (day_article>=today.day): return True
    if (today.year==year_article):
      if (today.month==month_article) and (today.day>=day_article): return True
      elif (today.month-1==month_article) and (today.day<=day_article): return True
    return False

In [ ]:
urls=['https://vnexpress.net/thoi-su','https://vnexpress.net/the-gioi','https://vnexpress.net/kinh-doanh','https://vnexpress.net/giai-tri','https://vnexpress.net/the-thao','https://vnexpress.net/phap-luat','https://vnexpress.net/giao-duc','https://vnexpress.net/suc-khoe','https://vnexpress.net/doi-song','https://vnexpress.net/du-lich','https://vnexpress.net/khoa-hoc','https://vnexpress.net/so-hoa','https://vnexpress.net/oto-xe-may','https://vnexpress.net/y-kien','https://vnexpress.net/tam-su','https://vnexpress.net/hai']
title_dates=[]
s=1
today=date.today()
for url in urls:
  #truy cập vào một chủ đề báo
  soup = get_page_content(url)
  print(url)
  satisfy=True
  while(satisfy==True):
    newspapers=soup.findAll('h3',class_='title-news')
    if (len(newspapers)==0): newspapers=soup.findAll('h2',class_='title-news')
    if (len(newspapers)==0): newspaper=soup.findAll('h2',class_='title_item')
    #titles=[newspaper.find('a').text for newspaper in newspapers] 
    links=[newspaper.find('a')['href'] for newspaper in newspapers]
    for link in links:
      string=[]
      #lay noi dung HTML tung bai bao 
      soup_link = get_page_content(link) 
      #lay title bai bao
      title=soup_link.find('h1',class_='title-detail')
      if (title!=None):
          title=title.text
          #lay the tag chua date
          date_tag=soup_link.find('span',class_="date") 
          if (date_tag==None): date_tag=soup_link.find('span',class_="time") 
          #tach ngay gio ra khoi tag 
          date=get_date(date_tag)  
          #nếu đạt điều kiện của hàm so sánh thì thực thi
          if (date_comparing(today,date)==True): 
            #tạo chuỗi thứ tự, title,ngày,giờ
             string=str(s)+", "+str(title)+" "+str(date)
             print(string)
             s+=1
          else: 
             satisfy=False
             break
    if satisfy!=False:
        nextpage_url=soup.find('a',class_="btn-page next-page ")
        print(nextpage_url)
        if (nextpage_url!=None): 
            nextpage_url=nextpage_url['href']
            if (url!='https://vnexpress.net/doi-song' and url!='https://vnexpress.net/du-lich' and url!='https://vnexpress.net/so-hoa/p2'):soup=get_page_content("https://vnexpress.net"+nextpage_url)
            else:  soup=get_page_content(nextpage_url)
        else: satisfy=False

https://vnexpress.net/thoi-su
1, TP HCM khánh thành hầm chui hơn 500 tỷ đồng Thứ bảy, 19/9/2020, 11:34 (GMT+7)
2, Cầu 2.250 tỷ đồng cuối sông Trà Khúc sắp hoàn thành Thứ bảy, 19/9/2020, 10:08 (GMT+7)
3, Cảnh sát đu dây giải cứu sáu người mắc kẹt trong nước lũ Thứ bảy, 19/9/2020, 09:18 (GMT+7)
4, Dùng lốp xe làm hộ lan trên đường cong Thứ bảy, 19/9/2020, 07:23 (GMT+7)
5, Xe tải đâm ôtô buýt, 20 người bị thương Thứ bảy, 19/9/2020, 01:56 (GMT+7)
6, Hàng chục tấn cam bị rụng Thứ bảy, 19/9/2020, 00:00 (GMT+7)
7, Bờ biển Cửa Đại sạt lở Thứ sáu, 18/9/2020, 23:26 (GMT+7)
8, Cháy hai tàu cá và ca nô Thứ sáu, 18/9/2020, 23:14 (GMT+7)
9, Bệnh viện K thí điểm tự chủ Thứ sáu, 18/9/2020, 23:09 (GMT+7)
10, Đường phố ở Hà Tĩnh, Vinh ngập hơn nửa mét Thứ sáu, 18/9/2020, 22:52 (GMT+7)
11, 7 huyện, thị xã ở Thừa Thiên Huế mất điện Thứ sáu, 18/9/2020, 21:27 (GMT+7)
12, Thủ tướng: 'Tiếp tục mở đường bay quốc tế có kiểm soát' Thứ sáu, 18/9/2020, 19:40 (GMT+7)
13, Khu dân cư ngập sâu một mét sau mưa lớn Thứ 